In [1]:
import librosa
# import librosa.display
# # import essentia.standard as es
# %matplotlib qt
# import matplotlib.pyplot as plt
from __future__ import print_function
import numpy as np

In [2]:
# this part is to test whether the wave file is loaded correctly
audio_file = 'vocal_short.wav'
audio, sr = librosa.load(audio_file, sr=48000, mono=True)
# plt.figure()
# plt.plot(audio)

In [3]:
onset_env = librosa.onset.onset_strength(audio, sr=sr)
tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)

In [4]:
print(tempo)

[122.2826087]


In [5]:
# data = vamp.collect(audio, sr, 'mtg-melodia:melodia')
# print(vamp.list_plugins)
# this part is to perform the stft
win_length = 1024
window = 'hann'
n_fft = 8192
hop_length = 128
X = librosa.stft(audio,n_fft = n_fft,hop_length = hop_length,\
                 win_length = win_length, window = window)
X_Normal = 2* np.absolute(X) / np.sum(np.hanning(win_length))
# plt.figure()
# librosa.display.specshow(librosa.amplitude_to_db(X_Normal,ref=np.max),\
#                          y_axis='log', x_axis='time')
# plt.colorbar(format='%+2.0f dB')

print(X_Normal.shape)

(4097, 8559)


In [6]:
freq_min = 55
freq_max = 1760
k_min = np.floor(freq_min * n_fft / sr)
k_max = np.floor(freq_max * n_fft / sr)
print(X_Normal.shape)
# print(X_Normal.shape)
X_Crop = X_Normal[int(k_min) : int(k_max),:]
print(X_Crop.shape)

# test librosa salient
freqs = np.linspace(freq_min,freq_max,num = X_Crop.shape[0])
# # freqs test ok
# # print(freqs)
# harms = [1,2,3,4,5]
# weights = [1.0,0.45,0.33,0.25,0.10]



# S_sal = librosa.salience(X_Crop,freqs,harms,weights,fill_value = 0)
# print(S_sal.shape)
# plt.figure()
# librosa.display.specshow(librosa.amplitude_to_db(S_sal,ref=np.max),sr = sr,\
#                          y_axis = 'linear', x_axis = 'time')


(4097, 8559)
(291, 8559)


In [7]:
from __future__ import division, print_function
import numpy as np

def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, show=False, ax=None):
    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan-1, indnan+1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size-1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind]-x[ind-1], x[ind]-x[ind+1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                    & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indices by their occurrence
        ind = np.sort(ind[~idel])
    return ind

In [8]:
sample_amount = X_Crop.shape[1]
# define the space to store the peak frequencies
pks_locs = np.zeros((sample_amount,3))

# import the signal package
from scipy import signal

for i in range(sample_amount):
    iFrame = X_Crop[:,i]
    peakInd = detect_peaks(iFrame)
    # plt.plot(X_Crop[:,850])
    # get the amplitude of the peaks
    if(peakInd.size >= 3):
        local_max_Peaks = iFrame[peakInd]
        # sort the amplitudes and find the stornges ones using sort
        indSort = np.argsort(local_max_Peaks)
        # find the frequency according to the index
        sortedLocs = peakInd[indSort]
        sortedLocs = sortedLocs[::-1]
        # put them into the memery space
        # just the first three strongest
        pks_locs[i,:] = sortedLocs[0:3]
    else:
        pks_locs[i,:] = np.array([0,0,0])



In [9]:
# this block is to seperate the seqs
hop = 128
frameSize = 1024

#now create a space to store the seqs
seqs = np.zeros((sample_amount,frameSize))
#Now we put the blocks into the seqs
for i in np.arange(0,len(audio)-frameSize,hop):
        seqs[i//128,:] = audio[i:i+frameSize]
        


In [10]:
# this block is to compute energy 
E = np.multiply(seqs,seqs)
E = np.sum(E,axis = 1)
E[np.where(E<0.0000001)] = 0.00001
E = np.log10(E)

# print(E.shape)

In [11]:
#this block is to compute zero crossing
zeros_cros = np.zeros((sample_amount,1))
for i in range(sample_amount):
    seq = seqs[i,:]
    indexs = np.diff(np.sign(seq))
    indexs = np.abs(indexs)
    zeros_pos = np.array((np.where(indexs == 2)))
    zeros_cros[i] = zeros_pos.shape[1]

In [12]:
# S_mfcc = librosa.feature.mfcc(y = audio,sr = sr, hop_length = hop_length, \
#                                         n_fft= n_fft,n_mfcc = 20)
auto_corr = np.zeros((sample_amount,1))
for i in range(sample_amount):
    seq = seqs[i,:]
    numerator = np.sum(np.multiply(seq[1:frameSize], seq[0:frameSize-1]))
    denominator = np.sqrt(np.sum(np.multiply(seq[1:frameSize],seq[1:frameSize]))  * \
                         np.sum(np.multiply(seq[0:frameSize-1],seq[0:frameSize-1])) )
    if(denominator == 0):
        denominator = 0.0001;
    
    auto_corr[i] = numerator / denominator

In [13]:
# transform all to np.array again
E = np.reshape(E,(sample_amount,1))
zeros_cros = np.reshape(zeros_cros,(sample_amount,1))
# print(zeros_cros.shape)
auto_corr = np.reshape(auto_corr,(sample_amount,1))
# print(auto_corr.shape)


pks_locs = np.array(pks_locs)
mean_pks = np.mean(pks_locs[:,0:2],axis = 1)
mean_pks = np.reshape(mean_pks,(sample_amount,1))
dataSet = np.concatenate((E,zeros_cros,auto_corr,mean_pks),axis = 1)



####################################################################################################
# pks_locs = pks_locs[:, 0:2]
# # after some experiments I found out it is better to sort the pks_locs somehow
# pks_locs.sort(axis=1)

# # concatenate along the features
# dataSet = np.concatenate((E, zeros_cros, auto_corr, pks_locs[:, 0:2]), axis=1)
#################################################################################################



# print(dataSet[1200:1250,:])
#zscore
from scipy import stats
dataSet = stats.zscore(dataSet,axis = 0)



In [14]:
from sklearn.cluster import KMeans
start_matrix = np.array([[1  ,-1  ,0  ,0  ],
                [0  ,0  ,1  ,-1   ],
                [-2  ,2  ,2  ,2   ],
                [-3  ,3  ,3  ,3   ]])
#####################################################################################

# start_matrix = np.array([[1, -1, 3, 0, 0],
#                          [0, 0, 2, -1, -1],
#                          [-2, 2, 1, 2, 1],
#                          [-3, 3, 0, 3, 3]])


##################################################################################


kmeans = KMeans(n_clusters = 4, init = start_matrix,n_init = 1).fit(dataSet)
pyidx = kmeans.labels_


# import scipy.io as sio
# sio.savemat('../MatlabExp/pyidx.mat',{'pyidx':pyidx})

## Above is finished run from the following

In [15]:
# %matplotlib qt
x = np.linspace(0,len(audio),len(audio))
x = x / 128;
# plt.figure()
pyidx[np.where(pyidx>0)] = 1
# plt.plot(x,audio)
diff_pyidx = np.diff(pyidx)
# plt.plot(diff_pyidx)



In [16]:

pitch_on = np.where(diff_pyidx <0)
pitch_off = np.where(diff_pyidx > 0)
# print(pitch_on[0])
# print(np.where(pitch_off > pitch_on[0,0]))
pitch_off_arr = pitch_off[0]
pitch_on_arr = pitch_on[0]
start = pitch_on_arr[0]
pitch_off_arr = pitch_off_arr[np.where(pitch_off_arr > start)]

print(pitch_on_arr)
print(pitch_off_arr)


[ 715  811  866  995 1078 1237 1248 1329 1533 1535 1584 1593 1745 2142
 2146 2151 2155 2168 2175 2194 2254 2266 2618 3148 3559 3565 3571 3579
 3623 3652 4003 4005 4370 4519 4530 4693 4704 4884 5045 5231 6278 6287
 6290 6294 6305 6342 6345 6386 6447 6515 7123 7308 7696 7706 7713 7716
 7719 7722 7724 7746 7829 7988 8117 8120 8122 8360]
[ 721  812  977 1060 1234 1247 1320 1516 1534 1536 1588 1724 2106 2144
 2148 2152 2164 2169 2184 2196 2261 2581 2957 3539 3560 3568 3572 3580
 3631 3964 4004 4303 4498 4524 4679 4698 4863 4987 5171 5845 6279 6289
 6293 6295 6307 6344 6352 6393 6499 7063 7268 7626 7705 7707 7714 7717
 7720 7723 7729 7751 7969 8077 8118 8121 8325 8550]


In [17]:
#find the duration of the small durations, here the threshold is set to 10
# that is approximately 30ms
duration = pitch_off_arr - pitch_on_arr
small_dur_idx = np.where(duration < 15)
pitch_on_arr = np.delete(pitch_on_arr,small_dur_idx)
pitch_off_arr = np.delete(pitch_off_arr,small_dur_idx)
print(pitch_on_arr)
print(pitch_off_arr)
print(len(pitch_on_arr))
# plt.figure()


[ 866  995 1078 1248 1329 1593 1745 2266 2618 3148 3652 4005 4370 4530
 4704 4884 5045 5231 6447 6515 7123 7308 7829 7988 8122 8360]
[ 977 1060 1234 1320 1516 1724 2106 2581 2957 3539 3964 4303 4498 4679
 4863 4987 5171 5845 6499 7063 7268 7626 7969 8077 8325 8550]
26


In [18]:
def findBreak(energy_wave,window_size = 80, hop_size = 10,height = 2.5):
    wave_dealt = energy_wave.flatten()
    wave_dealt =  wave_dealt[window_size//2: -window_size//2]
    wave_dealt = stats.zscore(wave_dealt)
    sample_amount = np.floor( (len(wave_dealt)-window_size)/hop_size)
    sample_amount = np.int16(sample_amount)
    sum_diff = np.zeros(sample_amount)
    for i in range(sample_amount):
        pos = i* hop_size
        wave_part = wave_dealt[pos : pos + window_size]
        diff_wave = np.diff(wave_part)
        sum_diff[i] = np.sum(diff_wave[0:window_size//2]) * (-1) \
                    + np.sum(diff_wave[window_size//2:])
#     plt.figure()
#     plt.plot(sum_diff)
#     plt.figure()
#     plt.plot(wave_dealt)
    from scipy.signal import find_peaks
    peaks,_ = find_peaks(sum_diff,height = height)
    final_peaks = np.array([])
    for peak in peaks:
        if(peak < sum_diff.size -1):
            if sum_diff[peak] > 3.5:
                final_peaks = np.append(final_peaks,peak)
            else:
                diff_area = sum_diff[peak-1] + sum_diff[peak+1]+sum_diff[peak]
                if diff_area > 6.0:
                    final_peaks = np.append(final_peaks,peak)
#     peaks,_ = find_peaks(sum_diff,height = height)
    i = 0
    if final_peaks.size >1:
        while i< final_peaks.size-1:
            if(final_peaks[i+1] - final_peaks[i])<=3:
                final_peaks = np.delete(final_peaks,i)
                i -= 1
            i+=1
    return final_peaks
        
    

In [19]:
# # # # plt.figure()
# # # # plt.plot(E[469+40:771-40].flatten())
# test = E[6791:7128].flatten()
# final_peaks = findBreak(test)
# print(final_peaks)
# window_size = 80
# hop_size = 10
# wave_dealt = stats.zscore(test)
# sample_amount = np.floor( (len(wave_dealt)-window_size)/hop_size)
# sample_amount = np.int16(sample_amount)
# sum_diff = np.zeros(sample_amount)
# for i in range(sample_amount):
#     pos = i* hop_size
#     wave_part = wave_dealt[pos : pos + window_size]
#     diff_wave = np.diff(wave_part)
#     sum_diff[i] = np.sum(diff_wave[0:window_size//2]) * (-1) \
#                 + np.sum(diff_wave[window_size//2:])
# plt.figure()
# plt.plot(sum_diff)
# peaks,_ = find_peaks(sum_diff,height = 2.5)
# final_peaks = np.array([])
# for peak in peaks:
#     if(peak < sum_diff.size -1):
#         diff_area = sum_diff[peak-1] + sum_diff[peak+1]+sum_diff[peak]
#         if diff_area > 6.0:
#             final_peaks = np.append(final_peaks,peak)

# print(final_peaks)
    
    
# print()

In [20]:
duration = pitch_off_arr - pitch_on_arr
big_dur_idx = np.where(duration > 215)
pitch_on_arr_big_dur = pitch_on_arr[big_dur_idx];
pitch_off_arr_big_dur = pitch_off_arr[big_dur_idx];
print(pitch_on_arr)
print(pitch_off_arr)
print(pitch_on_arr_big_dur)
print(pitch_off_arr_big_dur)
pitch_off_arr_inter= pitch_off_arr
pitch_on_arr_inter = pitch_on_arr
window_size = 80
hop_size = 10
for i in range(pitch_on_arr_big_dur.size):
    start = pitch_on_arr_big_dur[i]
    end = pitch_off_arr_big_dur[i]
    energy_wave = E[start:end]
    break_idx = findBreak(energy_wave)
    print(break_idx.size)
    if break_idx.size > 0:
        for i in range(break_idx.size):
            pitch_off_arr_inter = np.append(pitch_off_arr_inter,start +break_idx[i] * hop_size + window_size)
            pitch_on_arr_inter = np.append(pitch_on_arr_inter, start+ break_idx[i]* hop_size + window_size +window_size//2)

pitch_on_arr = np.sort(pitch_on_arr_inter)
pitch_off_arr = np.sort(pitch_off_arr_inter)

[ 866  995 1078 1248 1329 1593 1745 2266 2618 3148 3652 4005 4370 4530
 4704 4884 5045 5231 6447 6515 7123 7308 7829 7988 8122 8360]
[ 977 1060 1234 1320 1516 1724 2106 2581 2957 3539 3964 4303 4498 4679
 4863 4987 5171 5845 6499 7063 7268 7626 7969 8077 8325 8550]
[1745 2266 2618 3148 3652 4005 5231 6515 7308]
[2106 2581 2957 3539 3964 4303 5845 7063 7626]
0
1
1
1
2
1
0
2
0


In [21]:
%matplotlib qt
import matplotlib.pyplot as plt
# test boundaries
x = np.linspace(0,len(audio),len(audio))
x = x / 128;
plt.figure()
plt.plot(x,audio)
#for plot
boundary = np.zeros(np.int16(np.floor(len(audio)/128)))
boundary[np.int16(pitch_on_arr)] = 1
boundary[np.int16(pitch_off_arr)] = -1
plt.plot(boundary)

pitch_on_arr = np.int16(pitch_on_arr)
pitch_off_arr = np.int16(pitch_off_arr)

In [ ]:
# from scipy import stats
# pitch = stats.mode(pks_locs[397:445,0])
# pitch = pitch[0]
# freq_hz = 55 + (1760-55)/291 * pitch
# pitch_midi = 12 * np.log2(32 * freq_hz/440) + 9
# print(np.round(pitch_midi)[0])
# # print()

In [ ]:
import pretty_midi as pmidi
pm = pmidi.PrettyMIDI(initial_tempo = 80)
print(pm.instruments)
inst = pmidi.Instrument(program = 2, is_drum = False, name = 'piano')
pm.instruments.append(inst)
print(pm.instruments)
pks_locs.sort(axis = 1) 
for i in range(len(pitch_on_arr)):
    velocity = 95
    start = pitch_on_arr[i]
    end = pitch_off_arr[i]
    pitch = stats.mode(pks_locs[start:end,0])
#     print(pitch)
    pitch = pitch[0]
    freq_hz = 55 + (1760-55)/291 * pitch
    pitch_midi = 12 * np.log2(32 * freq_hz/440) + 9
#     print(pitch_midi)
    start_time = start /48000 * 128
    end_time = end/48000 * 128
    if (pitch_midi.size >0):
        inst.notes.append(pmidi.Note(velocity,np.int8(np.round(pitch_midi)[0]), start_time, end_time))
    else:
        inst.notes.append(pmidi.Note(velocity,0, start_time, end_time))
print(inst.notes)




In [ ]:
import IPython.display
fs = 16000 # I think this is the sampling rate for the music player to synthesize the music
IPython.display.Audio(pm.synthesize(fs = fs),rate = 16000)
# pm.write('test.mid')


In [ ]:
# print(pitch_on_arr)
# print(pitch_off_arr)
# for i in range(pitch_on_arr.size):
#     print(pks_locs[pitch_on_arr[i]:pitch_off_arr[i],:])
#     print('hello')
    
    
# a = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
# print

In [ ]:
# a = np.array([[1,2,3],[6,5,4],[7,8,9],[12,11,10]])
# print(a)
# a.sort(axis = 1)
# print(a)
# print(pks_locs)

print(pitch_on_arr)
print(pitch_off_arr)

In [ ]:
a = np.array([14,16,20,22])
# print(a)
# print(a[0])
i = 0
while i < (a.size-1):
    if a[i+1] - a[i] <=3:
        a = np.delete(a,i)
        print(a)
        i -= 1
    i +=1
    print(i)
print(a)

In [1]:
a = 'liyicheng.wav'
print(a[-4:])

.wav
